In [ ]:
import sys
import matplotlib.pyplot as plt
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
sys.path.append(r'reflutils')
from reflutils import FileDialog, Refl, tree

sns.set_style("white",rc = {"xtick.direction": "in", "ytick.direction": "in", "xtick.top":True, "ytick.right":True, "xtick.bottom" : True, "ytick.left", "": True, "grid.linestyle": "--"})
sns.set_context('notebook')
sns.set_palette('colorblind')

In [ ]:
df = pd.DataFrame({'x': [1, 2, 3], 'y': [10, 20, 30]})
fig, ax = plt.subplots()
ax.plot('x', 'y', data=df)
ax.set_title('title')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend(bbox_to_anchor=(1.02, .5), loc='upper left',borderaxespad=0)
plt.show()

In [ ]:
xrrDataBase = FileDialog.getDirectory(title="Select the XRR Database")

# tree(xrrDataBase)

In [ ]:
tree(xrrDataBase)

In [ ]:
energyList = xrrDataBase.iterdir()
energyList = [x for x in energyList if x.is_dir()]
POL = ["100.0", "190.0"]

In [ ]:
def multiPlot():
    combinedDataFrame = pd.DataFrame()
    wantedCols = ["Q", "Refl", "Err"]
    for e in energyList:
        try:
            xrr = Refl(e / POL[1])
            xrr.plot()
        except Exception as err:
            print(f"{e} - Could not plot {e.name} {POL[1]}")
            continue
        while True:
            if input("Press enter to continue") == "":
                break
            elif input("Press q to quit") == "q":
                return 0
        plt.plot()

In [168]:
multiPlot()

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
def multiEnPlot(pol :str = POL[0]):
    SCALE = -.5
    combinedDataFrame = pd.DataFrame()
    wantedCols = ["Q", "Refl", "Err"]
    fig, ax = plt.subplots(figsize=(6, 12))
    sns.color_palette("colorblind", n_colors=len(energyList))
    for i, e in enumerate(energyList):
        try:
            xrr = Refl(e / pol)
            combinedCols = [f"{colName}_{e.name}_{pol}" for colName in wantedCols]
            combinedDataFrame[combinedCols] = xrr.refl[wantedCols]

            x = xrr.refl["Q"]
            y = xrr.refl["Refl"]
            yerr = xrr.refl["Err"]

            ax.errorbar(x, y * 10 ** (SCALE * i), yerr=yerr * 10 ** (SCALE*i), label=f"{e.name} {pol}", fmt="o")
            ax.legend()
            ax.set_xlabel("Q")
            ax.set_ylabel("Reflectivity")
            ax.set_title("Reflectivity Curves \nCompared Across Energies", fontsize=16)
            ax.set_yscale("log")
            if i == 0:
                ax.set_xlim(0, max(x))
            else:
                if ax.get_xlim()[1] > max(x):
                    ax.set_xlim(0, max(x))
            ax.set_ylim(auto = True)
        except Exception as err:
            print(f"{err} - No data for  {e.name} {pol}")
            continue
    plt.show()
    return combinedDataFrame

In [ ]:
combinedRefl = multiEnPlot()

In [ ]:
# combinedRefl.to_csv(xrrDataBase / f"EnergyComparison_{POL[0]}.csv", index=False)
combinedRefl = pd.read_csv(xrrDataBase / f"EnergyComparison_{POL[0]}.csv")

In [ ]:
from re import S
from turtle import st

from matplotlib import axes


def justPlotting(df):
    # Plotting the data
    # fig, ax = plt.subplots(figsize=(6, 12))
    SCALE = -.5
    df = df.dropna()
    Q = df.columns[0]
    R = df.columns[1::3]
    Err = df.columns[2::3]
    fig, axes = plt.subplots(figsize=(4, 11))
    for i in range(len(R)):
        df.loc[:,R[i]] = df.loc[:,R[i]] * 10 ** (i*SCALE)
        df.loc[:,Err[i]] = df.loc[:,Err[i]] * 10 ** (i*SCALE)
        axes.errorbar(df.loc[:,Q], df.loc[:,R[i]], yerr=df.loc[:,Err[i]], fmt='.', label=R[i])
    axes.set_xlabel("Q")
    axes.set_ylabel('Reflectivity')
    axes.set_yscale('log')
    plt.legend()

justPlotting(combinedRefl)

In [ ]:
roomTemp = [pd.read_csv(csv).dropna(axis=1) for csv in xrrDataBase.glob("*.csv") if "RT" in csv.name]

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.set_palette("colorblind", n_colors=2)
Q250_100 = roomTemp[0]["Q_250.0_100.0"]
Q283p7_100 = roomTemp[0]["Q_283.7_100.0"]
R250_100 = roomTemp[0]["Refl_250.0_100.0"]
R283p7_100 = roomTemp[0]["Refl_283.7_100.0"]
Err250_100 = roomTemp[0]["Err_250.0_100.0"]
Err283p7_100 = roomTemp[0]["Err_283.7_100.0"]

Q250_190 = roomTemp[1]["Q_250.0_190.0"]
Q283p7_190 = roomTemp[1]["Q_283.7_190.0"]
R250_190 = roomTemp[1]["Refl_250.0_190.0"]
R283p7_190 = roomTemp[1]["Refl_283.7_190.0"]
Err250_190 = roomTemp[1]["Err_250.0_190.0"]
Err283p7_190 = roomTemp[1]["Err_283.7_190.0"]

ax.errorbar(Q250_100, R250_100*2, yerr=Err250_100*2, fmt='.-')
ax.errorbar(Q250_190, R250_190, yerr=Err250_190, fmt='.-')

ax.errorbar(Q283p7_100, R283p7_100*2e8, yerr=Err250_100*2e8, fmt='.-')
ax.errorbar(Q283p7_190, R283p7_190*1e8, yerr=Err250_190*1e8, fmt='.-')

plt.title("Reflectivity of Room Temp \nPrepared ZnPc Samples", fontsize=16)
ax.legend(["spol", "ppol"])
ax.hlines(5, 0, 0.25, linestyles='dashed', color = "grey")
ax.text(0,1e8,r"$\times 10^7$")
ax.text(.125,3e8,"283.7 eV", verticalalignment='center', horizontalalignment='center', fontsize=14)
ax.text(.125,1e0,"250 eV", verticalalignment='center', horizontalalignment='center', fontsize=14)

plt.semilogy()
plt.xlim(0,0.25)
plt.xlabel(r"Q $[/AA^{-1}]$")
plt.ylabel(r"Reflectivity")

In [ ]:
C60 = [pd.read_csv(csv).dropna(axis=1) for csv in xrrDataBase.glob("*.csv") if "RT" in csv.name]

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.set_palette("colorblind", n_colors=2)
Q250_100 = roomTemp[0]["Q_250.0_100.0"]
Q283p7_100 = roomTemp[0]["Q_283.7_100.0"]
R250_100 = roomTemp[0]["Refl_250.0_100.0"]
R283p7_100 = roomTemp[0]["Refl_283.7_100.0"]
Err250_100 = roomTemp[0]["Err_250.0_100.0"]
Err283p7_100 = roomTemp[0]["Err_283.7_100.0"]

Q250_190 = roomTemp[1]["Q_250.0_190.0"]
Q283p7_190 = roomTemp[1]["Q_283.7_190.0"]
R250_190 = roomTemp[1]["Refl_250.0_190.0"]
R283p7_190 = roomTemp[1]["Refl_283.7_190.0"]
Err250_190 = roomTemp[1]["Err_250.0_190.0"]
Err283p7_190 = roomTemp[1]["Err_283.7_190.0"]

ax.errorbar(Q250_100, R250_100*2, yerr=Err250_100*2, fmt='.-')
ax.errorbar(Q250_190, R250_190, yerr=Err250_190, fmt='.-')

ax.errorbar(Q283p7_100, R283p7_100*2e8, yerr=Err250_100*2e8, fmt='.-')
ax.errorbar(Q283p7_190, R283p7_190*1e8, yerr=Err250_190*1e8, fmt='.-')

plt.title("Reflectivity of Room Temp \nPrepared ZnPc Samples", fontsize=16)
ax.legend(["spol", "ppol"])
ax.hlines(5, 0, 0.25, linestyles='dashed', color = "grey")
ax.text(0,1e8,r"$\times 10^7$")
ax.text(.125,3e8,"283.7 eV", verticalalignment='center', horizontalalignment='center', fontsize=14)
ax.text(.125,1e0,"250 eV", verticalalignment='center', horizontalalignment='center', fontsize=14)

plt.semilogy()
plt.xlim(0,0.25)
plt.xlabel(r"Q $[/AA^{-1}]$")
plt.ylabel(r"Reflectivity")

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.set_palette("colorblind", n_colors=2)
Q250_100 = roomTemp[0]["Q_250.0_100.0"]
Q283p7_100 = roomTemp[0]["Q_283.7_100.0"]
R250_100 = roomTemp[0]["Refl_250.0_100.0"]
R283p7_100 = roomTemp[0]["Refl_283.7_100.0"]
Err250_100 = roomTemp[0]["Err_250.0_100.0"]
Err283p7_100 = roomTemp[0]["Err_283.7_100.0"]

Q250_190 = roomTemp[1]["Q_250.0_190.0"]
Q283p7_190 = roomTemp[1]["Q_283.7_190.0"]
R250_190 = roomTemp[1]["Refl_250.0_190.0"]
R283p7_190 = roomTemp[1]["Refl_283.7_190.0"]
Err250_190 = roomTemp[1]["Err_250.0_190.0"]
Err283p7_190 = roomTemp[1]["Err_283.7_190.0"]

ax.errorbar(Q250_100, R250_100*2, yerr=Err250_100*2, fmt='.-')
ax.errorbar(Q250_190, R250_190, yerr=Err250_190, fmt='.-')

ax.errorbar(Q283p7_100, R283p7_100*2e8, yerr=Err250_100*2e8, fmt='.-')
ax.errorbar(Q283p7_190, R283p7_190*1e8, yerr=Err250_190*1e8, fmt='.-')

plt.title("Reflectivity of Room Temp \nPrepared ZnPc Samples", fontsize=16)
ax.legend(["spol", "ppol"])
ax.hlines(5, 0, 0.25, linestyles='dashed', color = "grey")
ax.text(0,1e8,r"$\times 10^7$")
ax.text(.125,3e8,"283.7 eV", verticalalignment='center', horizontalalignment='center', fontsize=14)
ax.text(.125,1e0,"250 eV", verticalalignment='center', horizontalalignment='center', fontsize=14)

plt.semilogy()
plt.xlim(0,0.25)
plt.xlabel(r"Q $[/AA^{-1}]$")
plt.ylabel(r"Reflectivity")

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.set_palette("colorblind", n_colors=2)
Q250_100 = roomTemp[0]["Q_250.0_100.0"]
Q283p7_100 = roomTemp[0]["Q_283.7_100.0"]
R250_100 = roomTemp[0]["Refl_250.0_100.0"]
R283p7_100 = roomTemp[0]["Refl_283.7_100.0"]
Err250_100 = roomTemp[0]["Err_250.0_100.0"]
Err283p7_100 = roomTemp[0]["Err_283.7_100.0"]

Q250_190 = roomTemp[1]["Q_250.0_190.0"]
Q283p7_190 = roomTemp[1]["Q_283.7_190.0"]
R250_190 = roomTemp[1]["Refl_250.0_190.0"]
R283p7_190 = roomTemp[1]["Refl_283.7_190.0"]
Err250_190 = roomTemp[1]["Err_250.0_190.0"]
Err283p7_190 = roomTemp[1]["Err_283.7_190.0"]

ax.errorbar(Q250_100, R250_100*2, yerr=Err250_100*2, fmt='.-')
ax.errorbar(Q250_190, R250_190, yerr=Err250_190, fmt='.-')

ax.errorbar(Q283p7_100, R283p7_100*2e8, yerr=Err250_100*2e8, fmt='.-')
ax.errorbar(Q283p7_190, R283p7_190*1e8, yerr=Err250_190*1e8, fmt='.-')

plt.title("Reflectivity of Room Temp \nPrepared ZnPc Samples", fontsize=16)
ax.legend(["spol", "ppol"])
ax.hlines(5, 0, 0.25, linestyles='dashed', color = "grey")
ax.text(0,1e8,r"$\times 10^7$")
ax.text(.125,3e8,"283.7 eV", verticalalignment='center', horizontalalignment='center', fontsize=14)
ax.text(.125,1e0,"250 eV", verticalalignment='center', horizontalalignment='center', fontsize=14)

plt.semilogy()
plt.xlim(0,0.25)
plt.xlabel(r"Q $[/AA^{-1}]$")
plt.ylabel(r"Reflectivity")